In [1]:
# =====================================================
# BASELINE: SVM (LinearSVC) - FAKENEWSNET
# Output Format: HuggingFace Style
# =====================================================

import os, re, psutil, pickle, time
import pandas as pd
import numpy as np
from datasets import load_dataset, concatenate_datasets
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from google.colab import drive

# 1. MOUNT DRIVE
if not os.path.exists('/content/drive'):
    try:
        drive.mount('/content/drive', force_remount=True)
    except ValueError: pass

OUTPUT_DIR = "/content/drive/MyDrive/FakeNewsNet_SVM_Baseline"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 2. LOAD DATA (Gộp GossipCop + PolitiFact)
print("⏳ Đang tải dataset FakeNewsNet...")
try:
    # Cố gắng tải 2 tập con để có dữ liệu đầy đủ nhất
    ds_gossip = load_dataset("rickstello/FakeNewsNet", "gossipcop", split="train")
    ds_politi = load_dataset("rickstello/FakeNewsNet", "politifact", split="train")
    dataset_full = concatenate_datasets([ds_gossip, ds_politi])
    df = pd.DataFrame(dataset_full)
except Exception as e:
    print(f"⚠️ Tải config con thất bại ({e}), tải bản default...")
    dataset = load_dataset("rickstello/FakeNewsNet", split="train")
    df = pd.DataFrame(dataset)

print(f"Dataset columns: {df.columns.tolist()}") # In ra để kiểm tra
print(f"Tổng số mẫu dữ liệu: {len(df)}")

# 3. PRE-PROCESSING (FIX LỖI KEYERROR)

# A. Tìm tên cột an toàn
# Ưu tiên các tên cột thường gặp trong dataset này
text_col = next((c for c in ['news_content', 'text', 'content', 'body'] if c in df.columns), None)
title_col = next((c for c in ['title', 'news_title', 'headline'] if c in df.columns), None)
label_col = next((c for c in ['real', 'label', 'class', 'fake'] if c in df.columns), None)

if not label_col:
    raise ValueError(f"❌ Không tìm thấy cột nhãn! Các cột hiện có: {df.columns.tolist()}")

print(f"✅ Đã nhận diện cột: Text='{text_col}', Title='{title_col}', Label='{label_col}'")

# B. Xử lý dữ liệu an toàn
# Nếu không tìm thấy cột Text/Title thì dùng chuỗi rỗng thay vì để code crash
text_data = df[text_col].fillna('') if text_col else pd.Series([""] * len(df))
title_data = df[title_col].fillna('') if title_col else pd.Series([""] * len(df))

# Chuẩn hóa Label về int
df['label'] = df[label_col].astype(int)

# C. Clean Text
def clean_text_ml(s):
    if not isinstance(s, str): return ""
    s = s.lower()
    s = re.sub(r'https?://\S+', '', s)
    s = re.sub(r'<.*?>', '', s)
    s = re.sub(r'[^a-z0-9\s]', '', s)
    s = re.sub(r'\s+', ' ', s).strip()
    return s

print("🧹 Pre-processing...")
# Ghép Title + Text
df['content'] = (title_data + " " + text_data).apply(clean_text_ml)

# Lọc bỏ mẫu quá ngắn
df = df[df['content'].str.len() > 20]
print(f" → Sau xử lý: {len(df):,}")

# 4. SPLIT DATA
X_train_text, X_test_text, y_train, y_test = train_test_split(
    df['content'], df['label'], test_size=0.2, random_state=42, stratify=df['label']
)

print(f"Train: {len(X_train_text)} | Test: {len(X_test_text)}")

# 5. TF-IDF VECTORIZATION
print("\n⚙️ Vectorizing (TF-IDF)...")
# FakeNewsNet có bài dài, dùng 50k từ là hợp lý
vectorizer = TfidfVectorizer(max_features=50000, stop_words='english', ngram_range=(1, 2))

X_train = vectorizer.fit_transform(X_train_text)
X_test = vectorizer.transform(X_test_text)

# 6. TRAIN SVM (LinearSVC)
print("🚀 Training SVM (LinearSVC)...")

# Dùng LinearSVC cho tốc độ và hiệu suất cao trên text
# class_weight='balanced' rất quan trọng với FakeNewsNet (Real > Fake)
svm_model = LinearSVC(class_weight='balanced', random_state=42, max_iter=2000)

# Bọc trong CalibratedClassifierCV để tính được AUC (LinearSVC gốc không có predict_proba)
clf = CalibratedClassifierCV(svm_model)
clf.fit(X_train, y_train)

# =====================================================
# 7. EVALUATION
# =====================================================
print("\n🎯 ĐANG ĐÁNH GIÁ (TEST SET)...")

start_time = time.time()

# Dự đoán
y_pred = clf.predict(X_test)
y_prob = clf.predict_proba(X_test)[:, 1] # Xác suất lớp 1 (Real)

end_time = time.time()
runtime = end_time - start_time
samples_per_second = len(y_test) / runtime

# Metrics
accuracy = accuracy_score(y_test, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
auc = roc_auc_score(y_test, y_prob)

eval_results = {
    'eval_accuracy': accuracy,
    'eval_precision': precision,
    'eval_recall': recall,
    'eval_f1': f1,
    'eval_auc': auc,
    'eval_loss': 'N/A (SVM)',
    'eval_runtime': runtime,
    'eval_samples_per_second': samples_per_second,
    'eval_steps_per_second': 'N/A'
}

print("\n" + "="*50)
print("📊 KẾT QUẢ SVM BASELINE - FAKENEWSNET:")
print("="*50)
print(eval_results)
print("="*50)

# 8. SAVE
with open(os.path.join(OUTPUT_DIR, "svm_fnn_model.pkl"), "wb") as f:
    pickle.dump(clf, f)
with open(os.path.join(OUTPUT_DIR, "tfidf_fnn_vectorizer.pkl"), "wb") as f:
    pickle.dump(vectorizer, f)
print(f"\n✅ Đã lưu model tại: {OUTPUT_DIR}")

Mounted at /content/drive
⏳ Đang tải dataset FakeNewsNet...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

⚠️ Tải config con thất bại (BuilderConfig 'gossipcop' not found. Available: ['default']), tải bản default...


FakeNewsNet.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/23196 [00:00<?, ? examples/s]

Dataset columns: ['title', 'news_url', 'source_domain', 'tweet_num', 'real']
Tổng số mẫu dữ liệu: 23196
✅ Đã nhận diện cột: Text='None', Title='title', Label='real'
🧹 Pre-processing...
 → Sau xử lý: 22,355
Train: 17884 | Test: 4471

⚙️ Vectorizing (TF-IDF)...
🚀 Training SVM (LinearSVC)...

🎯 ĐANG ĐÁNH GIÁ (TEST SET)...

📊 KẾT QUẢ SVM BASELINE - FAKENEWSNET:
{'eval_accuracy': 0.842764482218743, 'eval_precision': 0.8377817144742215, 'eval_recall': 0.842764482218743, 'eval_f1': 0.8394047671549866, 'eval_auc': np.float64(0.8727983823123103), 'eval_loss': 'N/A (SVM)', 'eval_runtime': 0.02896738052368164, 'eval_samples_per_second': 154346.02367117154, 'eval_steps_per_second': 'N/A'}

✅ Đã lưu model tại: /content/drive/MyDrive/FakeNewsNet_SVM_Baseline
